#### *Closures*

In [2]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [3]:
a = averager()
a(10)

10.0

In [4]:
a(20)

15.0

In [5]:
a(30)

20.0

In [6]:
a.__closure__

(<cell at 0x0000019E42127340: list object at 0x0000019E425E6700>,)

In [7]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total, count
        total += number
        count += 1
        return total / count
    return add

In [8]:
a = averager()
a(10)

10.0

In [9]:
a(20)

15.0

In [10]:
a.__closure__

(<cell at 0x0000019E42127310: int object at 0x00007FFAF13523C8>,
 <cell at 0x0000019E42124700: int object at 0x00007FFAF1352748>)

In [11]:
from time import perf_counter
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() - start
    return poll

In [12]:
t1 = timer()

In [13]:
t1()

0.016775200027041137

In [14]:
t1()

0.040708999964408576

In [15]:
def counter(initial_value=0):
    def inc(increment=1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return inc

In [16]:
c1 = counter()
c1()

1


#### *Decorators*

In [17]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)
    return inner

In [18]:
def add(a, b):
    return a + b

In [19]:
def mult(a, b):
    return a * b

In [20]:
counter_add = counter(add)
counter_add.__closure__

(<cell at 0x0000019E42124790: int object at 0x00007FFAF1352388>,
 <cell at 0x0000019E42126C80: function object at 0x0000019E42592DE0>)

In [21]:
counter_add.__code__.co_freevars

('cnt', 'fn')

In [22]:
counter_add(10, 3)

add has been called 1 times


13

In [23]:
counter_add(50, 16)

add has been called 2 times


66

In [24]:
counter_mult = counter(mult)
counter_mult.__closure__

(<cell at 0x0000019E421279A0: int object at 0x00007FFAF1352388>,
 <cell at 0x0000019E42127730: function object at 0x0000019E42593BA0>)

In [25]:
counter_mult(10, 3)

mult has been called 1 times


30

In [26]:
counter_mult(11, 5)

mult has been called 2 times


55

In [27]:
def counter(fn, counters):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

In [28]:
c = dict()

In [29]:
counted_add = counter(add, c)
counted_mult = counter(mult, c)

In [30]:
counted_add(10, 3)

13

In [31]:
counted_add(16, 50)

66

In [32]:
counted_mult(10, 10)

100

In [33]:
counted_mult(11, 6)

66

In [34]:
c

{'add': 2, 'mult': 2}

In [35]:
add = counter(add, c)
mult = counter(mult, c)

In [36]:
add(1, 2)

3

In [37]:
mult(11, 3)

33

In [38]:
c

{'add': 1, 'mult': 1}

In [39]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)
    return inner

In [40]:
def add(a: int, b: int = 0):
    """
    adds two values
    """
    return a + b

In [41]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 0)
    adds two values



In [42]:
id(add)

1779229601088

In [43]:
add = counter(add)

In [44]:
id(add)

1779229601888

In [45]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [46]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)
    
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

In [47]:
@counter
def add(a: int, b: int = 0):
    """
    adds two values
    """
    return a + b

In [48]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    adds two values



In [49]:
def counter(fn):
    from functools import wraps
    cnt = 0
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'{fn.__name__} has been called {cnt} times')
        return fn(*args, **kwargs)
    return inner

In [50]:
@counter
def add(a: int, b: int = 0):
    """
    adds two values
    """
    return a + b

In [51]:
help(add)

Help on function add in module __main__:

add(a: int, b: int = 0)
    adds two values



In [52]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)

        print(f'{fn.__name__}({args_str}) took {elapsed}s to run')
        return result
    return inner

1. recursion

In [53]:
def calc_recursive_fib(n):
    if n <= 2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [54]:
@timed
def recursive_fib(n):
    return calc_recursive_fib(n) 

In [55]:
recursive_fib(35)

recursive_fib(35) took 1.559358699945733s to run


9227465

2. loop

In [56]:
@timed
def loop_fib(n):
    first, second = 1, 1
    for _ in range(3, n+1):
        first, second = second, first + second
    return second

In [57]:
loop_fib(35)

loop_fib(35) took 7.900060154497623e-06s to run


9227465

3. reduce

In [58]:
@timed
def reduce_fib(n):
    from functools import reduce
    dummy = range(n-1)
    initial = (1, 0)
    result = reduce(lambda prev, n: (prev[0]+prev[1], prev[0]), 
                    dummy, 
                    initial)
    return result[0]

In [59]:
reduce_fib(35)

reduce_fib(35) took 1.8999911844730377e-05s to run


9227465


#### *Decorators stacking*

In [60]:
def dec_1(fn):
    def inner():
        print("Running dec_1")
        return fn()
    return inner

In [61]:
def dec_2(fn):
    def inner():
        print("Running dec_2")
        return fn()
    return inner

In [62]:
@dec_1
@dec_2
def my_func():
    print("Running my_func")

my_func = dec_1(dec_2(my_func))

In [63]:
my_func()

Running dec_1
Running dec_2
Running my_func


In [64]:
def dec_1(fn):
    def inner():
        result = fn()
        print("Running dec_1")
        return result
    return inner

In [65]:
def dec_2(fn):
    def inner():
        result = fn()
        print("Running dec_2")
        return result
    return inner

In [66]:
@dec_1
@dec_2
def my_func():
    print("Running my_func")

In [67]:
my_func()

Running my_func
Running dec_2
Running dec_1



#### *Decorator application(Memoization)*


In [68]:
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-2) + fib(n-1)

In [69]:
fib(6)

Calculating fib(6)
Calculating fib(4)
Calculating fib(2)
Calculating fib(3)
Calculating fib(1)
Calculating fib(2)
Calculating fib(5)
Calculating fib(3)
Calculating fib(1)
Calculating fib(2)
Calculating fib(4)
Calculating fib(2)
Calculating fib(3)
Calculating fib(1)
Calculating fib(2)


8

In [70]:
# closure approach
def fib():
    cache = {1: 1, 2: 1}

    def calc_fib(n):
        if n not in cache:
            print(f'Calculating fib({n})')
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    return calc_fib

In [71]:
f = fib()
f(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


55

In [72]:
def memoize_fib(fib):
    cache = {1: 1, 2: 1}

    def inner(n):
        if n not in cache:
            cache[n] = fib(n)
        return cache[n]
    return inner

In [73]:
@memoize_fib
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [74]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


5

In [75]:
def memoize(fn):
    cache = dict()

    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    return inner

In [76]:
@memoize
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [77]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [78]:
fib(6)

Calculating fib(6)


8

In [79]:
@memoize
def fact(n):
    print(f"Calculating factorial of {n}")
    return 1 if n < 2 else n * fact(n-1)

In [80]:
fact(3)

Calculating factorial of 3
Calculating factorial of 2
Calculating factorial of 1


6

In [81]:
fact(4)

Calculating factorial of 4


24

In [82]:
from functools import lru_cache

In [83]:
@lru_cache()
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [84]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [85]:
fib(6)

Calculating fib(6)


8

In [86]:
@lru_cache(maxsize=5)
def fib(n):
    print(f'Calculating fib({n})')
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [87]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

In [88]:
fib(6)

Calculating fib(6)


8

In [89]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)


55


#### *Decorator factories*

In [90]:
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
       start = perf_counter()
       result = fn(*args, **kwargs)
       end = perf_counter()
       elapsed = end - start 
       print(f'Run time: {elapsed:.6f}')
       return result
    return inner

In [91]:
def calc_fib_recurse(n):
    return 1 if n < 3 else calc_fib_recurse(n-2) + calc_fib_recurse(n-1)

@timed
def fib(n):
    return calc_fib_recurse(n)

fib = timed(fib)

In [92]:
fib(10)

Run time: 0.000015


55

In [93]:
# hardcoded number of repetitions
def timed(fn):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for _ in range(10):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (end - start)
        avg_run_time =  total_elapsed / 10
        print(f'Avg Run time: {avg_run_time:.6f}')
        return result
    return inner

In [94]:
def fib(n):
    return calc_fib_recurse(n)

In [95]:
fib = timed(fib)

In [96]:
fib(10)

Avg Run time: 0.000009


55

In [97]:
def timed(fn, reps):
    from time import perf_counter

    def inner(*args, **kwargs):
        total_elapsed = 0
        for _ in range(reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (end - start)
        avg_run_time =  total_elapsed / reps
        print(f'Avg Run time: {avg_run_time:.6f} with {reps} reps')
        return result
    return inner

In [98]:
def fib(n):
    return calc_fib_recurse(n)

In [99]:
fib = timed(fib, 5) # but @timed(5) doesn't work

In [100]:
fib(10)

Avg Run time: 0.000010 with 5 reps


55

In [101]:
def dec(fn):
    print("running dec")

    def inner(*args, **kwargs):
        print("running inner")
        return fn(*args, **kwargs)
    
    return inner

In [102]:
@dec
def my_func():
    print("running my_func")

running dec


In [103]:
my_func()

running inner
running my_func


In [104]:
def dec_factory():
    print("running dec_factory")

    def dec(fn):
        print("running dec")

        def inner(*args, **kwargs):
            print("running inner")
            return fn(*args, **kwargs)
        
        return inner
    return dec

In [105]:
# my_func = dec_factory()(my_func)
@dec_factory()
def my_func():
    print("running my_func")

running dec_factory
running dec


In [106]:
def dec_factory(reps):    
    def timed(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for _ in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_run_time =  total_elapsed / reps
            print(f'Avg Run time: {avg_run_time:.8f} with {reps} reps')
            return result
        return inner
    return timed

In [107]:
@dec_factory(7)
def fib(n):
    return calc_fib_recurse(n)

In [108]:
fib(10)

Avg Run time: 0.00000944 with 7 reps


55

In [109]:
def timed(reps):    
    def dec(fn):
        from time import perf_counter

        def inner(*args, **kwargs):
            total_elapsed = 0
            for _ in range(reps):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                total_elapsed += (end - start)
            avg_run_time =  total_elapsed / reps
            print(f'Avg Run time: {avg_run_time:.8f} with {reps} reps')
            return result
        return inner
    return dec

In [110]:
@timed(13)
def fib(n):
    return calc_fib_recurse(n)

In [111]:
fib(10)

Avg Run time: 0.00000904 with 13 reps


55


#### *Decorator Class*


In [112]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, c):
        print(f"called a={self.a}, b={self.b}, c={c}")

In [113]:
obj = MyClass(10, 20)

In [114]:
obj(100)

called a=10, b=20, c=100


In [115]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, fn):
        def inner(*args, **kwargs):
            print(f"decorated function called a={self.a}, b={self.b}")
            return fn(*args, **kwargs)
        return inner

In [116]:
@MyClass(10, 20)
def my_func(s):
    print(f"Hello {s}")

In [117]:
my_func('World')

decorated function called a=10, b=20
Hello World



#### *Decorating Classes*


In [118]:
from fractions import Fraction

In [119]:
f = Fraction(2, 3)

In [120]:
f.numerator

2

In [121]:
f.denominator

3

In [122]:
# monkey patching
Fraction.speak = lambda self, message: f"Fraction says {message}"

In [123]:
f.speak('This is me')

'Fraction says This is me'

In [124]:
Fraction.is_integral = lambda self: self.denominator == 1

In [125]:
f1 = Fraction(2, 3)
f2 = Fraction(64, 8)

In [126]:
f1.is_integral()

False

In [127]:
f2.is_integral()

True

In [128]:
def dec_speak(cls):
    cls.speak = lambda self, message: f"{self.__class__.__name__} says {message}"
    return cls

In [129]:
Fraction = dec_speak(Fraction)

In [130]:
f1 = Fraction(2, 3)

In [131]:
f1.speak('hello')

'Fraction says hello'

In [132]:
from datetime import datetime, timezone

In [133]:
def info(self):
    results = []
    results.append(f'time: {datetime.now(timezone.utc)}')
    results.append(f'Class: {self.__class__.__name__}')
    results.append(f'id: {hex(id(self))}')
    for k, v in vars(self).items():
        results.append(f'{k}: {v}')
    return results

def debug_info(cls):
    cls.debug = info
    return cls 

In [134]:
# Person = debug_info(Person)
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    def say_hi(self):
        return 'Hello there'

In [135]:
p = Person('John', 1939)

In [136]:
p.debug()

['time: 2025-07-22 06:27:44.574331+00:00',
 'Class: Person',
 'id: 0x19e41ef2660',
 'name: John',
 'birth_year: 1939']


#### *Decorator Application: Single Dispatch Generic Functions*

In [137]:
from html import escape

In [138]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a: int):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a: float):
    return f'{round(a, 2)}'

def html_str(s: str):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l: list):
    items = (f'<li>{html_escape(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d: dict):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [139]:
print(html_str("""this is 
a multi line string
with special characters: 10 &lt; 100"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &amp;lt; 100


In [140]:
print(html_int(66))

66(<i>0x42</i>)


In [141]:
from decimal import Decimal

In [142]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [143]:
htmlize(6.6677)

'6.67'

In [144]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [145]:
# list elements are not htmlized
print(htmlize([(1, 2, 3), 6.6677, 100]))

<ul>
<li>(1, 2, 3)</li>
<li>6.6677</li>
<li>100</li>
</ul>


In [146]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a: int):
    return f'{a}(<i>{str(hex(a))}</i>)'

def html_real(a: float):
    return f'{round(a, 2)}'

def html_str(s: str):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l: list):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d: dict):
    items = (f'<li>{html_escape(k)}={htmlize(v)}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [147]:
print(htmlize([(1, 2, 3), 6.6677, 100]))

<ul>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
<li>3(<i>0x3</i>)</li>
</ul></li>
<li>6.67</li>
<li>100(<i>0x64</i>)</li>
</ul>


##### Better approach

In [148]:
def htmlize(arg):
    registry = {
        object: html_escape,
        int: html_int,
        float: html_real,
        Decimal: html_real,
        str: html_str,
        list: html_list,
        tuple: html_list,
        dict: html_dict
    }

    fn = registry.get(type(arg), registry[object])
    return fn(arg)

In [149]:
print(htmlize([(1, 2), 6.6677, 100]))

<ul>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
</ul></li>
<li>6.67</li>
<li>100(<i>0x64</i>)</li>
</ul>


In [150]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn

    def inner(arg):
        return registry[object](arg)

    return inner

In [151]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [152]:
htmlize('1 < 100')

'1 &lt; 100'

In [153]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn
    registry[int] = lambda a: f'{a}(<i>{str(hex(a))}</i>)'
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')

    def inner(arg):
        return registry.get(type(arg), registry[object])(arg)

    return inner

In [154]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [155]:
htmlize('1 < 100')

'1 &lt; 100'

In [156]:
htmlize(100)

'100(<i>0x64</i>)'

##### Making singledispatch more generic

In [192]:
def singledispatch(fn):
    registry = {}
    registry[object] = fn

    def decorated(arg):
        return registry.get(type(arg), registry[object])(arg)
    
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    def dispatch(type_):
        return registry.get(type_, registry[object])
    
    decorated.register = register
    decorated.registry = registry # not a good approach
    decorated.dispatch = dispatch

    return decorated

In [193]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [194]:
htmlize('1 < 100')

'1 &lt; 100'

In [195]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [196]:
htmlize(100)

'100(<i>0x64</i>)'

In [197]:
# html_sequence = html_register(tuple)(html_register(list)(html_sequence)) -->
@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l: list):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'


In [198]:
print(htmlize([(1, 2), 6.6677, 100]))

<ul>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
</ul></li>
<li>6.6677</li>
<li>100(<i>0x64</i>)</li>
</ul>


In [199]:
# now we also can htmlize tuple
print(htmlize(((1, 2), 6.6677, 100)))

<ul>
<li><ul>
<li>1(<i>0x1</i>)</li>
<li>2(<i>0x2</i>)</li>
</ul></li>
<li>6.6677</li>
<li>100(<i>0x64</i>)</li>
</ul>


In [200]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 list: <function __main__.html_sequence(l: list)>,
 tuple: <function __main__.html_sequence(l: list)>}

In [201]:
htmlize.dispatch(int)

<function __main__.html_int(a)>